# Processing airport changes during outbreak of COVID-19

### Load Python tools

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import geojson
import openpyxl
import altair as alt
from altair_saver import save
import lxml
import requests
from shapely.geometry import Point, LineString
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Read airport data

In [2]:
# https://datahub.io/core/airport-codes#data
airports = pd.read_csv('input/airport-codes_csv.csv', encoding='latin-1')

### Split coordinates

In [3]:
airports['coordinates'] = airports.coordinates.str.replace('(', '').str.replace(')', '')

lat = []
lon = []

for row in airports['coordinates']:
    try:
        lat.append(row.split(',')[1])
        lon.append(row.split(',')[0])
    except:
        lat.append(np.NaN)
        lon.append(np.NaN)

airports['latitude'] = lat
airports['longitude'] = lon
airports['latitude'] = airports['latitude'].astype(float)
airports['longitude'] = airports['longitude'].astype(float)

---

## Load flightradar24 airport data

In [4]:
cols_to_use = ['date', 'airport', 'scheduled', 'departed']

### USA flights

In [5]:
usa_src = pd.read_csv('./coronavirus/US-CAN Airports Scheduled vs Actual.csv',\
                      names=cols_to_use)

In [6]:
cali_airports = ['LAX', 'SAN', 'SFO']
can_airports = ['YUL', 'YVR', 'YYZ']

In [7]:
usa_src['can'] = usa_src['airport'].isin(can_airports)
usa_src['cali'] = usa_src['airport'].isin(cali_airports)

### European flights

In [8]:
europe_src = pd.read_csv('./coronavirus/European Departures Sched vs Actual 14Feb-26Apr.csv',\
                      header=0,names=cols_to_use)

## Basic calculations for flight differences then and now

### USA

In [9]:
usa_src['diff'] = usa_src['departed'] -  usa_src['scheduled']
usa_src['pct_diff'] = (((usa_src['departed'] - usa_src['scheduled'] )/\
                           usa_src['scheduled'])).round(2)

In [10]:
usa_src.head()

,date,airport,scheduled,departed,can,cali,diff,pct_diff
0,2/17/20,ATL,1159,1152,False,False,-7,-0.01
1,2/17/20,BOS,578,530,False,False,-48,-0.08
2,2/17/20,BWI,322,315,False,False,-7,-0.02
3,2/17/20,CLT,829,826,False,False,-3,-0.00
4,2/17/20,DEN,856,843,False,False,-13,-0.02


### Europe

In [11]:
europe_src['diff'] = europe_src['departed'] -  europe_src['scheduled']
europe_src['pct_diff'] = (((europe_src['departed'] - europe_src['scheduled'] )/\
                           europe_src['scheduled'])).round(2)

### Share of scheduled flights that didn't depart LAX, SFO

In [12]:
lax = usa_src[usa_src['airport'] == 'LAX']
sfo = usa_src[usa_src['airport'] == 'SFO']
san = usa_src[usa_src['airport'] == 'SAN']

In [13]:
lax.to_csv("output/lax_out.csv")
san.to_csv("output/san_out.csv")
sfo.to_csv("output/sfo_out.csv")

### Merge with airports list to get metadata about each one

In [14]:
usa_merge = usa_src.merge(airports, right_on='iata_code', left_on='airport', how='left')
europe_merge = europe_src.merge(airports, right_on='iata_code', left_on='airport', how='left')

---

## Aggregate change at European and American airports

In [15]:
d = dict.fromkeys(('scheduled', 'departed'), ['mean', 'sum'])

### Change in average daily flights and total flights in the United States

In [16]:
usa_agg = usa_merge.groupby('airport').agg(d).round(2).reset_index()

In [17]:
usa_agg.columns = ['_'.join(col) for col in usa_agg.columns.values]

In [18]:
usa_agg['sum_diff'] = usa_agg.departed_sum - usa_agg.scheduled_sum
usa_agg['pct_diff'] = (((europe_src['departed'] - europe_src['scheduled'] )/\
                           europe_src['scheduled'])).round(2)

### Change in average daily flights and total flights in the Europe

In [19]:
europe_agg = europe_src.groupby('airport').agg(d).round(2).reset_index()
europe_agg.columns = ['_'.join(col) for col in europe_agg.columns.values]

In [20]:
europe_agg['sum_diff'] = europe_agg.departed_sum - europe_agg.scheduled_sum
europe_agg['pct_diff'] = (((europe_src['departed'] - europe_src['scheduled'] )/\
                           europe_src['scheduled'])).round(2)

### Change by day in European flights

In [21]:
europe_date_agg = europe_merge.groupby('date').agg(d).round(2).reset_index()

In [22]:
europe_date_agg.columns = ['_'.join(col) for col in europe_date_agg.columns.values]

In [23]:
europe_date_agg['sum_diff'] = europe_date_agg.departed_sum - europe_date_agg.scheduled_sum
europe_date_agg['pct_diff'] = (((europe_date_agg['departed_sum'] - europe_date_agg['scheduled_sum'] )/\
                           europe_date_agg['scheduled_sum'])).round(2)

In [24]:
europe_date_agg_export = europe_date_agg[['date_', 'scheduled_sum', 'departed_sum', 'pct_diff']]

In [25]:
europe_date_agg_export.to_csv('output/europe_date_agg_export.csv')

In [26]:
europe_agg_chart = alt.Chart(europe_date_agg).mark_bar(size=4,color='#AAA79D').encode(
    x=alt.X('date_:T', title='', axis=alt.Axis(grid=False, tickCount=3, format='%b. %-d')),
    y=alt.Y('pct_diff:Q', title='', axis=alt.Axis(gridWidth=.6,\
    gridColor='#dddddd',offset=3,tickSize=0,domainOpacity=0,tickCount=4, format='%'), \
            scale=alt.Scale(domain=(-1, 0)))
).properties(width=500, height=350,
    title=''
)

(europe_agg_chart).configure_view(strokeOpacity=0)

alt.Chart(...)

In [27]:
europe_agg_chart_mobile = alt.Chart(europe_date_agg).mark_bar(size=3,color='#AAA79D').encode(
    x=alt.X('date_:T', title='', axis=alt.Axis(grid=False, tickCount=3, format='%b. %-d')),
    y=alt.Y('pct_diff:Q', title='', axis=alt.Axis(gridWidth=.6, gridColor='#dddddd',\
        offset=3,tickSize=0,domainOpacity=0,tickCount=4, format='%'), scale=alt.Scale(domain=(-1, 0)))
).properties(width=320, height=300,
    title=''
).configure(
    padding={"left": 0, "top": 20, "right": 0, "bottom": 0}
)

(europe_agg_chart_mobile).configure_view(strokeOpacity=0)

alt.Chart(...)

### Change by day in USA flights

In [28]:
usa_date_agg = usa_merge.groupby('date').agg(d).round(2).reset_index()

In [29]:
usa_date_agg.columns = ['_'.join(col) for col in usa_date_agg.columns.values]

In [30]:
usa_date_agg['sum_diff'] = usa_date_agg.departed_sum - usa_date_agg.scheduled_sum
usa_date_agg['pct_diff'] = (usa_date_agg['sum_diff']/
                        usa_date_agg.scheduled_sum).round(2)

In [31]:
usa_date_agg.sort_values(by='date_', ascending=False).head()

,date_,scheduled_mean,scheduled_sum,departed_mean,departed_sum,sum_diff,pct_diff
69,4/9/20,409.36,11462,199.64,5590,-5872,-0.51
68,4/8/20,403.25,11291,198.14,5548,-5743,-0.51
67,4/7/20,412.11,11539,203.96,5711,-5828,-0.51
66,4/6/20,420.46,11773,207.11,5799,-5974,-0.51
65,4/5/20,482.57,13512,207.21,5802,-7710,-0.57


In [32]:
usa_date_agg['date_'] = pd.to_datetime(usa_date_agg['date_'])

In [33]:
usa_date_agg_export = usa_date_agg[['date_', 'scheduled_sum', 'departed_sum', 'pct_diff']]

In [34]:
usa_agg_chart_mobile = alt.Chart(usa_date_agg).mark_bar(size=3,color='#C7733E').encode(
    x=alt.X('date_:T', title='', axis=alt.Axis(grid=False, tickCount=3, format='%b. %-d')),
    y=alt.Y('pct_diff:Q', title='', axis=alt.Axis(gridWidth=.6,\
    gridColor='#dddddd',offset=3,tickSize=0,domainOpacity=0,tickCount=4, format='%'), \
            scale=alt.Scale(domain=(-1, 0)))
).properties(width=320, height=300,
    title=''
).configure(
    padding={"left": 0, "top": 0, "right": 0, "bottom": 0}
)

(usa_agg_chart_mobile).configure_view(strokeOpacity=0)

alt.Chart(...)

In [35]:
usa_date_agg_export.to_csv('output/usa_date_agg_export.csv')

In [36]:
usa_agg_chart = alt.Chart(usa_date_agg).mark_bar(size=4,color='#C7733E').encode(
    x=alt.X('date_:T', title='', axis=alt.Axis(grid=False, tickCount=3, format='%b. %-d')),
    y=alt.Y('pct_diff:Q', title='', axis=alt.Axis(gridWidth=.6,\
    gridColor='#dddddd',offset=3,tickSize=0,domainOpacity=0,tickCount=4, format='%'), \
            scale=alt.Scale(domain=(-1, 0)))
).properties(width=500, height=350,
    title=''
)

(usa_agg_chart|europe_agg_chart).configure(
    padding={"left": 10, "top": 0, "right": -3, "bottom": 0}
).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
)

alt.HConcatChart(...)

In [37]:
save((usa_agg_chart|europe_agg_chart).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa_europe_agg_flights_desktop.png', \
     scale_factor=2.0)

In [38]:
save((usa_agg_chart|europe_agg_chart).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
), \
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa_europe_agg_flights_desktop.svg', \
     scale_factor=2.0)

In [39]:
save((usa_agg_chart).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
), \
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa_agg_chart_desktop.png', \
     scale_factor=2.0)

In [40]:
save((usa_agg_chart_mobile).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
), \
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/agg_flights_mobile_us.png', \
     scale_factor=2.0)
save((europe_agg_chart_mobile).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=12
), \
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/agg_flights_mobile_eur.png', \
     scale_factor=3.0)

In [41]:
europe_merge.iloc[0]

date                                     2/14/20
airport                                      ARN
scheduled                                    289
departed                                     285
diff                                          -4
pct_diff                                   -0.01
ident                                       ESSA
type                               large_airport
name                   Stockholm-Arlanda Airport
elevation_ft                                 137
continent                                     EU
iso_country                                   SE
iso_region                                 SE-AB
municipality                           Stockholm
gps_code                                    ESSA
iata_code                                    ARN
local_code                                   NaN
coordinates     17.918600082397, 59.651901245117
latitude                                 59.6519
longitude                                17.9186
Name: 0, dtype: obje

### Small multiples for U.S. airports

In [42]:
alt.Chart(usa_merge).mark_bar(color='#C7733E', size=1.3).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(tickCount=2, format='%b. %-d', grid=False)),
        y=alt.Y('pct_diff:Q', title='', scale=alt.Scale(domain=(-1, 0)), axis=alt.Axis(tickSize=0,domainOpacity=0,\
                                                tickCount=4, format='%',gridWidth=.6, gridColor='#dddddd')),
    color=alt.condition(
        alt.datum.can == True,
        alt.value('#AAA79D'), 
        alt.value('#C4C1B3')
    )
).properties(width=100, height=100).facet(
    facet=alt.Facet('airport:N', title=''),
    columns=7,
    title='Daily reduction in scheduled flights, by U.S. airport'
).configure_view(strokeOpacity=0)

alt.FacetChart(...)

--- 

## Loop and cut little charts for each airport in the US and Europe

### Cut individual USA charts with no axis for desktop

In [43]:
usa_chart_no_y = []

for a in usa_src['airport'].unique():
    data = pd.DataFrame(usa_src[usa_src['airport'] == a])
    for i in data:
        usa_chart_no_y.append(alt.Chart(data).mark_bar(size=1.1,color='#C7733E').encode(
        x=alt.X('date:T', title='', axis=alt.Axis(tickSize=0, domainOpacity=0,labelColor='#ffffff',tickCount=3, \
                                                  format='%b. %-d',grid=False)),
        y=alt.Y('pct_diff:Q', title=None, axis=alt.Axis(labelColor='#ffffff',tickCount=4,tickSize=0, 
                            domainOpacity=0, format='%',gridWidth=.6, gridColor='#dddddd'),
        scale=alt.Scale(domain=(-1, 0)))
            ,
    color=alt.condition(
        alt.datum.cali == True,
        alt.value('#C7733E'), 
        alt.value('#C4C1B3')
    )
).configure(
    padding={"left": -45, "top": 0, "right": 0, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0))
    for c in usa_chart_no_y:
        save(c, '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/{}_noaxis.png'.format(a), scale_factor=2.0)

### Cut individual USA charts with no axis for mobile

In [44]:
usa_chart_no_y = []

for a in usa_src['airport'].unique():
    data = pd.DataFrame(usa_src[usa_src['airport'] == a])
    for i in data:
        usa_chart_no_y.append(alt.Chart(data).mark_bar(size=1.1,color='#C7733E').encode(
        x=alt.X('date:T', title='', axis=alt.Axis(tickSize=0, domainOpacity=0,labelColor='#ffffff',tickCount=3, \
                                                  format='%b. %-d',grid=False)),
        y=alt.Y('pct_diff:Q', title=None, axis=alt.Axis(labelColor='#ffffff',tickCount=4,tickSize=0, 
                            domainOpacity=0, format='%',gridWidth=.6, gridColor='#dddddd'),
        scale=alt.Scale(domain=(-1, 0)))
            ,
    color=alt.condition(
        alt.datum.cali == True,
        alt.value('#C7733E'), 
        alt.value('#C4C1B3')
    )
).configure(
    padding={"left": -40, "top": 0, "right": 0, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0))
    for c in usa_chart_no_y:
        save(c, '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/{}_noaxis_mobile.png'.format(a), scale_factor=2.0)

### Just LAX for annotation

In [45]:
laxchart = alt.Chart(lax).mark_bar(size=1.1).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(tickSize=0, domainOpacity=0, labelColor='#ffffff', \
                                              tickCount=3, format='%b. %-d',grid=False)),
    y=alt.Y('pct_diff:Q', title=None, axis=alt.Axis(labelColor='#ffffff',tickCount=4,tickSize=0, 
                            domainOpacity=0, format='%',gridWidth=.6, gridColor='#dddddd'),
        scale=alt.Scale(domain=(-1, 0))),
    color=alt.condition(
        alt.datum.cali == True,
        alt.value('#C7733E'), 
        alt.value('#C4C1B3')
    ),
)

laxtext = (
    alt.Chart(lax.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=8)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

laxtextmobile = (
    alt.Chart(lax.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=14)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

In [46]:
sanchart = alt.Chart(san).mark_bar(size=1.1).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(tickSize=0, domainOpacity=0, labelColor='#ffffff', \
                                              tickCount=3, format='%b. %-d',grid=False)),
    y=alt.Y('pct_diff:Q', title=None, axis=alt.Axis(labelColor='#ffffff',tickCount=4,tickSize=0, 
                            domainOpacity=0, format='%',gridWidth=.6, gridColor='#dddddd'),
        scale=alt.Scale(domain=(-1, 0))),
    color=alt.condition(
        alt.datum.cali == True,
        alt.value('#C7733E'), 
        alt.value('#C4C1B3')
    ),
)

santext = (
    alt.Chart(san.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=8)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

santextmobile = (
    alt.Chart(san.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=14)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

In [47]:
sfochart = alt.Chart(sfo).mark_bar(size=1.1).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(tickSize=0, domainOpacity=0, labelColor='#ffffff', \
                                              tickCount=3, format='%b. %-d',grid=False)),
    y=alt.Y('pct_diff:Q', title=None, axis=alt.Axis(labelColor='#ffffff',tickCount=4,tickSize=0, 
                            domainOpacity=0, format='%',gridWidth=.6, gridColor='#dddddd'),
        scale=alt.Scale(domain=(-1, 0))),
    color=alt.condition(
        alt.datum.cali == True,
        alt.value('#C7733E'), 
        alt.value('#C4C1B3')
    ),
)

sfotext = (
    alt.Chart(sfo.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=8)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

sfotextmobile = (
    alt.Chart(sfo.query("pct_diff == pct_diff.min()"))
    .mark_text(dy=10, color="#000000", font='Benton Gothic', fontWeight='bolder', fontSize=14)
    .encode(x=alt.X("date:T"), y=alt.Y("pct_diff:Q"), text=alt.Text("pct_diff:Q",format=',.0%'))
)

In [48]:
save((laxchart + laxtext).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/LAX_noaxis.png', scale_factor=2.0)

save((laxchart + laxtextmobile).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/LAX_noaxis_mobile.png', scale_factor=2.0)

In [49]:
save((sfochart + sfotext).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/SFO_noaxis.png', scale_factor=2.0)

save((sfochart + sfotextmobile).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/SFO_noaxis_mobile.png', scale_factor=2.0)

In [50]:
save((sanchart + santext).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/SAN_noaxis.png', scale_factor=2.0)

save((sanchart + santextmobile).configure(
    padding={"left": -45, "top": 0, "right": -3, "bottom": 0}
).properties(width=100, height=100,
    title='').configure_view(strokeOpacity=0),\
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/usa/SAN_noaxis_mobile.png', scale_factor=2.0)

---

### TSA rider data

In [51]:
url = 'https://www.tsa.gov/coronavirus/passenger-throughput'

In [52]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
  Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [53]:
r = requests.get(url, headers=header)

In [54]:
tsa_dfs = pd.read_html(r.text)
tsa_tables = pd.DataFrame(tsa_dfs[0])

In [55]:
tsa_tables.rename(columns={"0":"date","1":"2020 Travelers","2":"2019 Travelers"}, inplace=True)
tsa_tables.columns = ["date", "2020", "2019"]

In [56]:
tsa_tables = tsa_tables.iloc[1:]

In [57]:
tsa_tables['date'] = pd.to_datetime(tsa_tables['date'])

In [58]:
tsa_tables.head(10)

,date,2020,2019
1,2020-04-27,119854,2412770
2,2020-04-26,128875,2506809
3,2020-04-25,114459,1990464
4,2020-04-24,123464,2521897
5,2020-04-23,111627,2526961
6,2020-04-22,98968,2254209
7,2020-04-21,92859,2227475
8,2020-04-20,99344,2594171
9,2020-04-19,105382,2356802
10,2020-04-18,97236,1988205


In [59]:
tsa_tables_melt = pd.melt(tsa_tables, id_vars=['date'], value_vars=['2020', '2019'],
        var_name='year', value_name='travelers')

In [60]:
tsa_tables_melt['mark'] = 'Mar. 13: National emergency declaration'

In [61]:
tsa_tables.to_csv('output/tsa_tables.csv')

In [62]:
tsa_tables_melt.to_csv('output/tsa_tables_melt.csv')

In [63]:
tsa_desktop_chart = alt.Chart(tsa_tables_melt).mark_line(size=4).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(offset=15, tickCount=4, format='%b. %-d', grid=False)),
    y=alt.Y('travelers:Q', title='', axis=alt.Axis(tickSize=0,domainOpacity=0,\
                                    tickCount=4,offset=3, gridWidth=.6, gridColor='#dddddd', format='.1s')),
    color=alt.Color('year',
                   scale=alt.Scale(
            domain=['2020', '2019'],
            range=['#C7733E', '#C4C1B3']), legend=None),
)

tsa_desktop_text = (
    alt.Chart(tsa_tables_melt.query("date == '2020-03-13' & year == '2020'"))
    .mark_text(dy=-20, dx=142, angle=0,color="#000000", font='Benton Gothic', fontSize=13)
    .encode(x=alt.X("date:T"), y=alt.Y("travelers:Q"), text=alt.Text("mark"))
)

(tsa_desktop_chart + tsa_desktop_text).configure(
    padding={"left": -5, "top": 0, "right": 0, "bottom": 0}
).configure_axis(
    labelFontSize=14
).configure_view(strokeOpacity=0).properties(width=600, height=400,
    title='')

alt.LayerChart(...)

In [64]:
save((tsa_desktop_chart + tsa_desktop_text)\
     .configure_view(strokeOpacity=0).properties(width=600, height=400,
    title=''), \
     '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/tsa_desktop.png', scale_factor=2.0)

In [65]:
tsa_mobile_chart = alt.Chart(tsa_tables_melt).mark_line(size=3).encode(
    x=alt.X('date:T', title='', axis=alt.Axis(offset=15, tickCount=4, format='%b. %-d', grid=False)),
    y=alt.Y('travelers:Q', title='', axis=alt.Axis(tickSize=0,domainOpacity=0,\
                                    tickCount=6,offset=3, gridWidth=.6, gridColor='#dddddd', format='.1s')),
    color=alt.Color('year',
                   scale=alt.Scale(
            domain=['2020', '2019'],
            range=['#C7733E', '#C4C1B3']), legend=None),
)

tsa_mobile_text = (
    alt.Chart(tsa_tables_melt.query("date == '2020-03-13' & year == '2020'"))
    .mark_text(dy=-10, dx=125, angle=0,color="#000000", font='Benton Gothic', fontSize=13)
    .encode(x=alt.X("date:T"), y=alt.Y("travelers:Q"), text=alt.Text("mark"))
)

(tsa_mobile_chart + tsa_mobile_text).properties(width=300, height=300,
    title='').configure(
    padding={"left": 0, "top": 0, "right": 0, "bottom": 0}
).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
)

alt.LayerChart(...)

In [66]:
save((tsa_mobile_chart + tsa_mobile_text).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
).properties(width=300, height=300,
    title=''), '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/tsa_mobile.png', scale_factor=3.0)
save((tsa_mobile_chart + tsa_mobile_text).configure_view(strokeOpacity=0).configure_axis(
    labelFontSize=13
).properties(width=300, height=300,
    title=''), '/Users/mhustiles/data/github/coronavirus-flight-viz/assets/images/tsa_mobile.svg', scale_factor=3.0)